# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




---

## **Create Tables**

In [8]:
session_query = """CREATE TABLE IF NOT EXISTS session_songs 
(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))
"""

user_songs_query = """CREATE TABLE IF NOT EXISTS user_songs 
(user_id int, session_id int, artist text, song_title text, item_in_session int, first_name text, last_name text,
PRIMARY KEY ((user_id, session_id), item_in_session))
"""

music_app_history_query = """CREATE TABLE IF NOT EXISTS music_app_history (song_title text, first_name text, last_name text,
PRIMARY KEY (song_title))
"""
try:
    session.execute(session_query)
    session.execute(user_songs_query)
    session.execute(music_app_history_query)
except Exception as e:
    print(e)

## **Load tables**

* Read all CSV's from event_data folder
* Create a new CSV with all registers
* Load tables

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        
        # Insert into session_songs table
        insert_session_query = "INSERT INTO session_songs (session_id, item_in_session, artist, song_title, song_length)"
        insert_session_query = insert_session_query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(insert_session_query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))
        
        # Insert into user_songs table
        insert_user_songs_query = "INSERT INTO user_songs (user_id, session_id, artist, song_title, item_in_session, first_name, last_name)"
        insert_user_songs_query = insert_user_songs_query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(insert_user_songs_query, (int(userId), int(sessionId), artist_name, song, int(itemInSession), user_name, user_last_name))
        
        # Insert into music_app_history table
        insert_music_app_history_query = "INSERT INTO music_app_history (song_title, first_name, last_name)"
        insert_music_app_history_query = insert_music_app_history_query + " VALUES (%s, %s, %s)"
        session.execute(insert_music_app_history_query, (song, user_name, user_last_name))

## SELECT to verify that the data have been inserted into each table

#### **Query 1**:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [34]:
df_session_songs = pd.DataFrame()
try:
    query = "SELECT artist, song_title, song_length FROM session_songs WHERE session_id = 338 AND item_in_session = 4";
    rows = session.execute(query)
    
    df_session_songs = pd.DataFrame(list(rows))
except Exception as e:
    print(e)

df_session_songs.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### **Query 2**: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [36]:
df_user_songs = pd.DataFrame()
try:
    query = "SELECT artist, song_title, first_name, last_name, item_in_session FROM user_songs WHERE user_id = 10 AND session_id = 182";
    rows = session.execute(query)
    
    df_user_songs = pd.DataFrame(list(rows))
except Exception as e:
    print(e)

df_user_songs.head()

,artist,song_title,first_name,last_name,item_in_session
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
1,Three Drives,Greece 2000,Sylvie,Cruz,1
2,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [37]:
df_music_app_history = pd.DataFrame()
try:
    query = "SELECT first_name, last_name FROM music_app_history WHERE song_title = 'All Hands Against His Own'";
    rows = session.execute(query)
    
    df_music_app_history = pd.DataFrame(list(rows))
except Exception as e:
    print(e)

df_music_app_history.head()       

,first_name,last_name
0,Jacqueline,Lynch


### Drop the tables before closing out the sessions

In [38]:
session.execute("DROP TABLE session_songs")
session.execute("DROP TABLE user_songs")
session.execute("DROP TABLE music_app_history")

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()